In [51]:
input_path_sample = "/share/students/bigdata/Dati/Lab6/Reviews.csv"

In [8]:
from pyspark import SparkConf, SparkContext

In [56]:
def pairs_finder(row):
    values=[]
    for el1 in row:
        for el2 in row:
            if el1<el2:
                value = ((el1, el2), 1)
                values.append(value)
    return values
                
if __name__ == "__main__":
    conf = SparkConf().setAppName("Amazon Reviews")
    context = SparkContext.getOrCreate(conf = conf)

    reviews = context.textFile(input_path_sample)

    reviews_without_first_row = reviews.filter(lambda row: row.startswith("Id,") == False)

    key_values_rdd = reviews_without_first_row.map(lambda row: (row.split(",")[2], row.split(",")[1]))

    key_values_rdd = key_values_rdd.groupByKey().mapValues(lambda value: list(set(value)))

    key_values_rdd = key_values_rdd.map(lambda row: row[1])

    pairs = key_values_rdd.flatMap(pairs_finder)
    
    pairs = pairs.reduceByKey(lambda v1, v2: v1+v2)

    pairs = pairs.filter(lambda v: v[1] > 1)

    pairs_sorted = pairs.sortBy(lambda v: v[1], ascending=False)
    pairs_sorted.saveAsTextFile("output")
    print(pairs_sorted.take(10))
   
    

[Stage 111:>                                                        (0 + 9) / 9]

[(('B002QWP89S', 'B002QWP8H0'), 608), (('B0026RQTGE', 'B002QWHJOU'), 608), (('B002QWHJOU', 'B002QWP89S'), 608), (('B0026RQTGE', 'B002QWP8H0'), 608), (('B0026RQTGE', 'B002QWP89S'), 608), (('B002QWHJOU', 'B002QWP8H0'), 608), (('B000KV61FC', 'B000KV7ZGQ'), 555), (('B001RVFERK', 'B007M832YY'), 551), (('B0013NUGDE', 'B0026KNQSA'), 551), (('B0013NUGDE', 'B001RVFERK'), 551)]
